In [60]:
import pandas as pd
import numpy as np

for personality in ['A', 'C', 'E', 'O', 'N']:
    filename = 'data/CPED_' + personality +'.tsv'#
#     filename = 'data/Friends_' + personality +'_whole.tsv'
    df_test = pd.read_csv(filename, sep='\t')
    # df_test = pd.read_csv('data/CPED_' + personality +'.tsv', sep='\t')

    ## get vad dict
    VAD_Lexicons = pd.read_csv('data/NRC-VAD-Lexicon-Aug2018Release/NRC-VAD-Lexicon.txt', sep='\t')
    VAD_dict = {}
    for r in VAD_Lexicons.iterrows():
        VAD_dict[r[1]['Word']] = [r[1]['Valence'], r[1]['Arousal'], r[1]['Dominance']]

    def vad_to_big_five(V, A, D):
        E = 0.23 * V + 0.12 * A + 0.82 * D 
        A = 0.83 * V + 0.19 * A - 0.21 * D
        C = 0.32 * V + 0.00 * A + 0.30 * D
        N = 0.57 * V - 0.65 * A + 0.00 * D
        O = 0.00 * V - 0.33 * A + 0.67 * D

        return [A, C, E, O, N]

    from nltk import word_tokenize, pos_tag
    from nltk.corpus import wordnet
    from nltk.stem import WordNetLemmatizer

    wnl = WordNetLemmatizer()

    def get_wordnet_pos(tag):
        if tag.startswith('J'):
            return wordnet.ADJ
        elif tag.startswith('V'):
            return wordnet.VERB
        elif tag.startswith('N'):
            return wordnet.NOUN
        elif tag.startswith('R'):
            return wordnet.ADV
        else:
            return None


    def get_personality_vad(sentence):

        tokens = word_tokenize(sentence)  # 分词
        tagged_sent = pos_tag(tokens)     # 获取单词词性
        lemmas_sent = []
        for tag in tagged_sent:
            wordnet_pos = get_wordnet_pos(tag[1]) or wordnet.NOUN
            lemmas_sent.append(wnl.lemmatize(tag[0], pos=wordnet_pos)) # 词形还原

        sent = [i.lower() for i in lemmas_sent]
        sent_v, sent_a, sent_d = 0,0,0

        word_cnt = 0
        for word in sent:
            try:
                sent_v += VAD_dict[word][0]
                sent_a += VAD_dict[word][1]
                sent_d += VAD_dict[word][2]
                word_cnt += 1
            except:
                pass
        try:
            sent_v /= word_cnt
            sent_a /= word_cnt
            sent_d /= word_cnt
        except:
            pass
            # print(sent)
        # print(sent_v, sent_a, sent_d)
        return vad_to_big_five(sent_v, sent_a, sent_d)

    df_test['VAD_personality'] = df_test['utterance'].apply(get_personality_vad)
    df_test.to_csv(data/CPED_' + personality +'.tsv, sep='\t', index=False)
    print(filename)

    # from sklearn.metrics import f1_score, accuracy_score

    # for threshold in [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
    #     df_test['VAD_label_'+personality] = df_test['VAD_personality'].apply(lambda x: 1 if x[0] >= threshold else 0)
    #     cnt = 0
    #     pos_cnt = 0
    #     neg_cnt = 0
    #     for i,r in df_test.iterrows():
    #         if r['VAD_label_'+personality] == r['labels']:
    #             cnt += 1
    #             if r['labels'] == 1:
    #                 pos_cnt += 1
    #             elif r['labels'] == 0:
    #                 neg_cnt += 1

    #     print('Threshold is: ', threshold)
    #     print('F1: ', f1_score(df_test['labels'], df_test['VAD_label_'+personality]), ' ACC: ', accuracy_score(df_test['labels'], df_test['VAD_label_'+personality]))
    # #     print('Positive: ', pos_cnt, ', Neagtive: ', neg_cnt, ' Total correct: ', cnt, 'Size: ', df_test.shape[0], cnt/df_test.shape[0])


KeyError: 'utterance'

In [35]:
df_test['labels'].sum()

405